In [9]:
import pandas as pd
import numpy as np
import pmdarima as pm
import warnings
import matplotlib.pyplot as plt # 可选，用于绘图

warnings.filterwarnings("ignore") # 忽略 ARIMA 拟合中可能出现的警告

# 1. 准备数据 (包含所有批次的数据)
# --- 更新 data 字典 ---
data = {
    '年份': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
    # 第1批
    '北京': [105.77, 114.81, 123.23, 122.35, 108.59, 102.95, 102.90, 105.52, 110.17, 103.46, 95.26, 86.09],
    '天津': [91.30, 89.88, 94.47, 103.68, 109.48, 113.04, 117.02, 124.33, 130.42, 130.90, 131.46, 131.84],
    '河北': [81.48, 83.65, 82.64, 94.45, 101.51, 106.19, 111.22, 116.77, 117.40, 114.08, 113.52, 116.03],
    '山西': [106.53, 97.57, 86.21, 86.05, 84.72, 85.81, 86.35, 92.07, 93.91, 93.60, 93.59, 94.86],
    # 第2批
    '内蒙古': [72.29, 71.91, 70.61, 74.37, 78.32, 82.76, 89.48, 93.14, 94.75, 91.87, 90.72, 90.92],
    '辽宁': [103.02, 104.08, 95.14, 93.12, 90.92, 98.01, 105.61, 107.50, 110.64, 109.81, 111.02, 107.74],
    '吉林': [77.11, 73.96, 72.69, 76.36, 79.74, 88.03, 96.87, 107.23, 105.96, 99.26, 96.07, 97.28],
    '黑龙江': [88.53, 90.84, 74.73, 80.20, 86.93, 95.15, 101.55, 107.77, 105.56, 98.37, 93.57, 89.78],
    '上海': [195.87, 199.82, 193.81, 181.52, 172.33, 174.85, 164.24, 145.55, 124.86, 107.03, 98.66, 94.89],
    '江苏': [83.99, 81.35, 80.91, 85.81, 90.82, 96.50, 101.85, 108.33, 114.54, 118.61, 122.16, 123.76],
    '浙江': [95.03, 95.18, 96.93, 99.74, 98.58, 100.72, 102.85, 101.99, 98.31, 95.85, 97.84, 99.69],
    '安徽': [85.50, 73.37, 71.22, 73.42, 79.42, 87.93, 95.32, 98.07, 99.10, 98.24, 98.99, 97.66],
    '福建': [93.20, 96.78, 98.96, 92.71, 91.27, 91.61, 95.92, 100.46, 104.50, 109.12, 113.10, 114.58],
    '江西': [91.68, 90.74, 83.99, 86.68, 90.21, 94.98, 103.69, 115.24, 123.49, 122.90, 119.55, 114.30],
    # 第3批
    '山东': [121.32, 110.07, 105.51, 105.12, 103.49, 103.36, 103.89, 104.77, 107.04, 111.80, 120.15, 123.80],
    '河南': [119.02, 115.10, 98.15, 103.26, 107.33, 113.30, 120.59, 131.85, 140.76, 144.97, 149.03, 154.16],
    '湖北': [76.88, 65.12, 69.80, 75.08, 85.13, 96.42, 104.18, 110.41, 112.28, 112.53, 114.12, 115.98],
    '湖南': [87.93, 87.41, 91.02, 95.44, 96.64, 97.00, 98.33, 103.43, 107.88, 110.59, 113.10, 113.49],
    '广东': [132.39, 126.82, 118.09, 108.50, 101.99, 100.56, 103.51, 107.68, 111.10, 113.86, 117.76, 121.94],
    '广西': [87.69, 88.24, 90.82, 94.67, 98.77, 100.95, 103.36, 108.07, 111.08, 112.09, 112.72, 115.99],
    '海南': [90.19, 85.91, 85.10, 86.79, 87.93, 88.40, 91.60, 97.08, 101.23, 103.81, 106.05, 108.69],
    '重庆': [102.75, 100.41, 101.57, 106.55, 114.16, 121.60, 124.25, 125.97, 133.25, 137.74, 135.04, 127.64],
    '四川': [81.00, 79.10, 76.08, 79.77, 84.60, 93.11, 102.90, 113.31, 119.91, 122.47, 123.51, 123.87],
    '贵州': [89.62, 86.83, 86.88, 87.79, 88.32, 90.12, 93.71, 99.87, 106.60, 112.67, 119.83, 130.30],
    '云南': [80.81, 77.35, 75.27, 77.48, 78.58, 79.47, 82.45, 86.30, 91.38, 96.33, 101.33, 105.39],
    '西藏': [74.32, 70.98, 69.00, 69.93, 69.96, 76.07, 82.03, 86.08, 93.34, 96.17, 98.87, 105.35],
    '陕西': [96.26, 88.81, 86.26, 86.47, 88.89, 90.21, 90.20, 93.59, 98.10, 103.18, 106.50, 111.76],
    '甘肃': [95.20, 92.64, 85.75, 85.41, 85.01, 86.11, 87.14, 90.39, 91.84, 89.78, 89.34, 90.00],
    '青海': [69.02, 64.55, 65.40, 68.36, 70.75, 71.31, 73.23, 82.55, 86.22, 88.09, 88.71, 92.20],
    '宁夏': [99.90, 98.09, 97.06, 97.05, 97.08, 97.99, 99.57, 103.29, 105.57, 103.56, 103.08, 103.77],
    '新疆': [97.27, 92.13, 89.93, 91.16, 93.39, 94.14, 95.66, 97.02, 100.66, 104.38, 108.85, 112.66],
    '大连市': [127.53, 127.86, 125.98, 122.01, 123.20, 123.97, 130.33, 133.51, 141.03, 141.90, 138.07, 134.99],
    '宁波市': [132.53, 128.27, 128.12, 130.14, 139.07, 145.67, 149.69, 147.43, 153.23, 158.37, 161.93, 163.06],
    '厦门市': [112.40, 112.22, 114.87, 219.25, 227.30, 243.44, 254.68, 253.30, 250.15, 246.08, 237.01, 230.31],
    # 第4批 (本次新增)
    '青岛市': [111.81, 110.71, 113.06, 113.91, 114.40, 115.71, 118.40, 120.25, 121.36, 124.85, 130.45, 138.36],
    '深圳市': [597.57, 603.29, 578.65, 534.73, 482.72, 460.55, 474.07, 490.73, 484.91, 436.45, 402.13, 382.65]
}
# --- 结束更新 data 字典 ---

df = pd.DataFrame(data)
df['年份'] = pd.to_datetime(df['年份'], format='%Y')
df.set_index('年份', inplace=True)
# 将索引转换为 PeriodIndex，更适合年度数据
df.index = df.index.to_period('A')

# 2. 定义预测参数
# locations 列表会自动更新，因为它是从 data 字典动态生成的
locations = list(data.keys())[1:]
forecast_horizon = 2035 - 2021 # 预测 14 年 (2022 到 2035)

# 3. 循环为每个地区/城市进行 ARIMA 预测
forecast_results = {}
fitted_models = {}

print(f"共 {len(locations)} 个地区/城市需要处理。")
print("开始为每个地区/城市拟合 ARIMA 模型并预测...\n")

for i, loc in enumerate(locations):
    print(f"--- 正在处理: {loc} ({i+1}/{len(locations)}) ---")
    series = df[loc]

    try:
        # 使用 auto_arima 自动寻找最佳模型参数
        auto_model = pm.auto_arima(series,
                                   start_p=0, start_q=0,
                                   max_p=3, max_q=3, # 限制模型复杂度
                                   m=1,              # 年度数据，非季节性
                                   d=None,           # 让 auto_arima 自动确定 d
                                   test='adf',       # 使用 ADF 检验确定 d
                                   seasonal=False,   # 明确指定非季节性
                                   stepwise=True,    # 使用 stepwise 算法加速搜索
                                   suppress_warnings=True, 
                                   error_action='ignore', # 忽略不合适的模型
                                   trace=False)       # 不打印搜索过程中的每一步

        if auto_model is None or not hasattr(auto_model, 'order'):
             raise ValueError("auto_arima未能找到合适的模型。")

        print(f"为 '{loc}' 找到的最佳 ARIMA 模型阶数: {auto_model.order}")

        # 打印模型详细参数
        print("模型详细参数:")
        print(auto_model.summary())
        print("-" * 60) # 添加分隔线

        fitted_models[loc] = auto_model # 保存拟合好的模型

        # 进行预测
        forecast, conf_int = auto_model.predict(n_periods=forecast_horizon, return_conf_int=True)

        # 将预测结果添加到字典
        forecast_results[loc] = forecast

    except Exception as e:
        print(f"!!! 无法为 '{loc}' 拟合或预测 ARIMA 模型: {e}")
        print("   可能是因为数据序列过短、模式不明显、数值过大/波动剧烈或auto_arima搜索失败。")
        # 如果模型失败，用 NaN 填充预测值
        forecast_results[loc] = [np.nan] * forecast_horizon
        fitted_models[loc] = None
    finally:
        print("\n") # 每个地区处理完后加一个空行

# 4. 整理并展示预测结果
forecast_index = pd.period_range(start=str(df.index[-1].year + 1), periods=forecast_horizon, freq='A')
forecast_df = pd.DataFrame(forecast_results, index=forecast_index)
forecast_df_formatted = forecast_df.round(2).astype(str) + '%'
forecast_df_formatted = forecast_df_formatted.replace('nan%', 'N/A')

print("\n\n--- ARIMA 模型预测结果 (2022-2035) ---")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(forecast_df_formatted)
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

# 5. 可选：绘制历史数据和预测结果
plot_results = False # 设置为 False 则不绘图

if plot_results:
    print("\n--- 正在绘制结果图表 ---")
    n_locations = len(locations)
    cols_per_row = 3
    n_rows = (n_locations + cols_per_row - 1) // cols_per_row
    fig_width = 15
    fig_height_per_row = 4
    fig_height = n_rows * fig_height_per_row

    fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(fig_width, fig_height), sharex=True, squeeze=False)
    fig.suptitle('各地区/城市比例历史数据与 ARIMA 预测 (2010-2035)', fontsize=16, y=0.99)

    try:
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
    except Exception as font_error:
        print(f"\n注意: 设置中文字体失败: {font_error}")
        print("图表标题和标签可能显示乱码。")

    axes_flat = axes.flatten()

    for i, loc in enumerate(locations):
        ax = axes_flat[i]
        # 绘制历史数据
        if loc in df.columns:
             df[loc].plot(ax=ax, label='历史数据', marker='o', linestyle='-')

        # 绘制预测数据
        if loc in forecast_df.columns and forecast_df[loc].notna().any():
            forecast_df[loc].plot(ax=ax, label='ARIMA 预测', linestyle='--', marker='x')

            # 绘制置信区间
            if fitted_models.get(loc) is not None:
                try:
                    _, conf_int_plot = fitted_models[loc].predict(n_periods=forecast_horizon, return_conf_int=True)
                    conf_int_df = pd.DataFrame(conf_int_plot, index=forecast_index, columns=['lower', 'upper'])
                    # 对于数值范围特别大的序列(如深圳)，置信区间可能非常宽，甚至出现负值
                    # 可以考虑对置信区间下限进行限制，例如不小于0
                    conf_int_df['lower'] = conf_int_df['lower'].clip(lower=0)

                    ax.fill_between(conf_int_df.index.to_timestamp(),
                                    conf_int_df['lower'],
                                    conf_int_df['upper'],
                                    color='gray', alpha=0.2, label='95% 置信区间 (≥0)')
                except Exception as plot_ci_error:
                     print(f"绘制 {loc} 的置信区间时出错: {plot_ci_error}")

        ax.set_title(loc)
        ax.set_ylabel('比例 (%)')
        ax.legend(fontsize='small')
        ax.grid(True, linestyle=':', alpha=0.6)
        # 对于数值范围差异巨大的序列（如深圳 vs 其他），Y轴自动缩放可能使其他图难以辨认
        # 可以考虑为特定序列设置不同的Y轴，但这会使比较困难
        # 或者接受Y轴自动缩放，关注每个图的自身趋势
        # ax.set_ylim(bottom=0) # 强制Y轴从0开始

    # 隐藏多余的子图
    for j in range(i + 1, len(axes_flat)):
        axes_flat[j].set_visible(False)

    # 调整X轴标签
    for row in range(n_rows):
        for col in range(cols_per_row):
            ax = axes[row, col]
            if ax.is_visible():
                if row == n_rows - 1:
                    ax.set_xlabel('年份')
                else:
                    ax.tick_params(labelbottom=False)

    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.show()

共 36 个地区/城市需要处理。
开始为每个地区/城市拟合 ARIMA 模型并预测...

--- 正在处理: 北京 (1/36) ---
为 '北京' 找到的最佳 ARIMA 模型阶数: (0, 2, 0)
模型详细参数:
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   12
Model:               SARIMAX(0, 2, 0)   Log Likelihood                 -33.683
Date:                Wed, 26 Mar 2025   AIC                             69.366
Time:                        12:07:58   BIC                             69.669
Sample:                    12-31-2010   HQIC                            69.034
                         - 12-31-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2        49.3411     26.923      1.833      0.067      -3.427     102.109
Ljung-Box (L1) (Q)

In [18]:
import pandas as pd
import numpy as np
import pmdarima as pm
import warnings
import matplotlib.pyplot as plt # 可选，用于绘图
import os # 导入 os 模块用于文件路径操作

warnings.filterwarnings("ignore") # 忽略 ARIMA 拟合中可能出现的警告

# ... (代码的第 1, 2, 3 部分保持不变) ...

# 1. 准备数据 (包含所有批次的数据)
# --- 更新 data 字典 ---
data = {
    '年份': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
    # 第1批
    '北京': [105.77, 114.81, 123.23, 122.35, 108.59, 102.95, 102.90, 105.52, 110.17, 103.46, 95.26, 86.09],
    '天津': [91.30, 89.88, 94.47, 103.68, 109.48, 113.04, 117.02, 124.33, 130.42, 130.90, 131.46, 131.84],
    '河北': [81.48, 83.65, 82.64, 94.45, 101.51, 106.19, 111.22, 116.77, 117.40, 114.08, 113.52, 116.03],
    '山西': [106.53, 97.57, 86.21, 86.05, 84.72, 85.81, 86.35, 92.07, 93.91, 93.60, 93.59, 94.86],
    # 第2批
    '内蒙古': [72.29, 71.91, 70.61, 74.37, 78.32, 82.76, 89.48, 93.14, 94.75, 91.87, 90.72, 90.92],
    '辽宁': [103.02, 104.08, 95.14, 93.12, 90.92, 98.01, 105.61, 107.50, 110.64, 109.81, 111.02, 107.74],
    '吉林': [77.11, 73.96, 72.69, 76.36, 79.74, 88.03, 96.87, 107.23, 105.96, 99.26, 96.07, 97.28],
    '黑龙江': [88.53, 90.84, 74.73, 80.20, 86.93, 95.15, 101.55, 107.77, 105.56, 98.37, 93.57, 89.78],
    '上海': [195.87, 199.82, 193.81, 181.52, 172.33, 174.85, 164.24, 145.55, 124.86, 107.03, 98.66, 94.89],
    '江苏': [83.99, 81.35, 80.91, 85.81, 90.82, 96.50, 101.85, 108.33, 114.54, 118.61, 122.16, 123.76],
    '浙江': [95.03, 95.18, 96.93, 99.74, 98.58, 100.72, 102.85, 101.99, 98.31, 95.85, 97.84, 99.69],
    '安徽': [85.50, 73.37, 71.22, 73.42, 79.42, 87.93, 95.32, 98.07, 99.10, 98.24, 98.99, 97.66],
    '福建': [93.20, 96.78, 98.96, 92.71, 91.27, 91.61, 95.92, 100.46, 104.50, 109.12, 113.10, 114.58],
    '江西': [91.68, 90.74, 83.99, 86.68, 90.21, 94.98, 103.69, 115.24, 123.49, 122.90, 119.55, 114.30],
    # 第3批
    '山东': [121.32, 110.07, 105.51, 105.12, 103.49, 103.36, 103.89, 104.77, 107.04, 111.80, 120.15, 123.80],
    '河南': [119.02, 115.10, 98.15, 103.26, 107.33, 113.30, 120.59, 131.85, 140.76, 144.97, 149.03, 154.16],
    '湖北': [76.88, 65.12, 69.80, 75.08, 85.13, 96.42, 104.18, 110.41, 112.28, 112.53, 114.12, 115.98],
    '湖南': [87.93, 87.41, 91.02, 95.44, 96.64, 97.00, 98.33, 103.43, 107.88, 110.59, 113.10, 113.49],
    '广东': [132.39, 126.82, 118.09, 108.50, 101.99, 100.56, 103.51, 107.68, 111.10, 113.86, 117.76, 121.94],
    '广西': [87.69, 88.24, 90.82, 94.67, 98.77, 100.95, 103.36, 108.07, 111.08, 112.09, 112.72, 115.99],
    '海南': [90.19, 85.91, 85.10, 86.79, 87.93, 88.40, 91.60, 97.08, 101.23, 103.81, 106.05, 108.69],
    '重庆': [102.75, 100.41, 101.57, 106.55, 114.16, 121.60, 124.25, 125.97, 133.25, 137.74, 135.04, 127.64],
    '四川': [81.00, 79.10, 76.08, 79.77, 84.60, 93.11, 102.90, 113.31, 119.91, 122.47, 123.51, 123.87],
    '贵州': [89.62, 86.83, 86.88, 87.79, 88.32, 90.12, 93.71, 99.87, 106.60, 112.67, 119.83, 130.30],
    '云南': [80.81, 77.35, 75.27, 77.48, 78.58, 79.47, 82.45, 86.30, 91.38, 96.33, 101.33, 105.39],
    '西藏': [74.32, 70.98, 69.00, 69.93, 69.96, 76.07, 82.03, 86.08, 93.34, 96.17, 98.87, 105.35],
    '陕西': [96.26, 88.81, 86.26, 86.47, 88.89, 90.21, 90.20, 93.59, 98.10, 103.18, 106.50, 111.76],
    '甘肃': [95.20, 92.64, 85.75, 85.41, 85.01, 86.11, 87.14, 90.39, 91.84, 89.78, 89.34, 90.00],
    '青海': [69.02, 64.55, 65.40, 68.36, 70.75, 71.31, 73.23, 82.55, 86.22, 88.09, 88.71, 92.20],
    '宁夏': [99.90, 98.09, 97.06, 97.05, 97.08, 97.99, 99.57, 103.29, 105.57, 103.56, 103.08, 103.77],
    '新疆': [97.27, 92.13, 89.93, 91.16, 93.39, 94.14, 95.66, 97.02, 100.66, 104.38, 108.85, 112.66],
    '大连市': [127.53, 127.86, 125.98, 122.01, 123.20, 123.97, 130.33, 133.51, 141.03, 141.90, 138.07, 134.99],
    '宁波市': [132.53, 128.27, 128.12, 130.14, 139.07, 145.67, 149.69, 147.43, 153.23, 158.37, 161.93, 163.06],
    '厦门市': [112.40, 112.22, 114.87, 219.25, 227.30, 243.44, 254.68, 253.30, 250.15, 246.08, 237.01, 230.31],
    # 第4批
    '青岛市': [111.81, 110.71, 113.06, 113.91, 114.40, 115.71, 118.40, 120.25, 121.36, 124.85, 130.45, 138.36],
    '深圳市': [597.57, 603.29, 578.65, 534.73, 482.72, 460.55, 474.07, 490.73, 484.91, 436.45, 402.13, 382.65]
}
df = pd.DataFrame(data)
df['年份'] = pd.to_datetime(df['年份'], format='%Y')
df.set_index('年份', inplace=True)
df.index = df.index.to_period('A')

# 2. 定义预测参数
locations = list(data.keys())[1:]
forecast_horizon = 2035 - 2021

# 3. 循环为每个地区/城市进行 ARIMA 预测
forecast_results = {}
fitted_models = {}

print(f"共 {len(locations)} 个地区/城市需要处理。")
print("开始为每个地区/城市拟合 ARIMA 模型并预测...\n")

# ... (ARIMA 拟合和预测的循环代码不变) ...
for i, loc in enumerate(locations):
    print(f"--- 正在处理: {loc} ({i+1}/{len(locations)}) ---")
    series = df[loc]
    try:
        auto_model = pm.auto_arima(series, start_p=0, start_q=0, max_p=3, max_q=3, m=1, d=None, test='adf',
                                   seasonal=False, stepwise=True, suppress_warnings=True, error_action='ignore', trace=False)
        if auto_model is None or not hasattr(auto_model, 'order'):
             raise ValueError("auto_arima未能找到合适的模型。")
        print(f"为 '{loc}' 找到的最佳 ARIMA 模型阶数: {auto_model.order}")
        print("模型详细参数:")
        print(auto_model.summary())
        print("-" * 60)
        fitted_models[loc] = auto_model
        forecast, conf_int = auto_model.predict(n_periods=forecast_horizon, return_conf_int=True)
        forecast_results[loc] = forecast
    except Exception as e:
        print(f"!!! 无法为 '{loc}' 拟合或预测 ARIMA 模型: {e}")
        print("   可能是因为数据序列过短、模式不明显、数值过大/波动剧烈或auto_arima搜索失败。")
        forecast_results[loc] = [np.nan] * forecast_horizon
        fitted_models[loc] = None
    finally:
        print("\n")


# 4. 整理并展示预测结果
forecast_index = pd.period_range(start=str(df.index[-1].year + 1), periods=forecast_horizon, freq='A')
forecast_df = pd.DataFrame(forecast_results, index=forecast_index)
forecast_df_formatted = forecast_df.round(2).astype(str) + '%'
forecast_df_formatted = forecast_df_formatted.replace('nan%', 'N/A')

print("\n\n--- ARIMA 模型预测结果 (2022-2035) ---")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(forecast_df_formatted)
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

# --- 新增：导出到 CSV ---
output_filename = 'arima_forecast_2022_2035.csv'
try:
    # 使用 utf-8-sig 编码确保 Excel 正确识别中文和特殊字符
    forecast_df_formatted.to_csv(output_filename, encoding='utf-8-sig')
    print(f"\n预测结果已成功导出到文件: {os.path.abspath(output_filename)}") # 打印完整路径
except Exception as e:
    print(f"\n!!! 导出 CSV 文件时出错: {e}")
# --- 结束新增部分 ---


# 5. 可选：绘制历史数据和预测结果
plot_results = False # 设置为 False 则不绘图

if plot_results:
    print("\n--- 正在绘制结果图表 ---")
    # ... (绘图代码不变) ...
    n_locations = len(locations)
    cols_per_row = 3
    n_rows = (n_locations + cols_per_row - 1) // cols_per_row
    fig_width = 15
    fig_height_per_row = 4
    fig_height = n_rows * fig_height_per_row

    fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(fig_width, fig_height), sharex=True, squeeze=False)
    fig.suptitle('各地区/城市比例历史数据与 ARIMA 预测 (2010-2035)', fontsize=16, y=0.99)

    try:
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
    except Exception as font_error:
        print(f"\n注意: 设置中文字体失败: {font_error}")
        print("图表标题和标签可能显示乱码。")

    axes_flat = axes.flatten()

    for i, loc in enumerate(locations):
        ax = axes_flat[i]
        if loc in df.columns:
             df[loc].plot(ax=ax, label='历史数据', marker='o', linestyle='-')
        if loc in forecast_df.columns and forecast_df[loc].notna().any():
            forecast_df[loc].plot(ax=ax, label='ARIMA 预测', linestyle='--', marker='x')
            if fitted_models.get(loc) is not None:
                try:
                    _, conf_int_plot = fitted_models[loc].predict(n_periods=forecast_horizon, return_conf_int=True)
                    conf_int_df = pd.DataFrame(conf_int_plot, index=forecast_index, columns=['lower', 'upper'])
                    conf_int_df['lower'] = conf_int_df['lower'].clip(lower=0)
                    ax.fill_between(conf_int_df.index.to_timestamp(),
                                    conf_int_df['lower'],
                                    conf_int_df['upper'],
                                    color='gray', alpha=0.2, label='95% 置信区间 (≥0)')
                except Exception as plot_ci_error:
                     print(f"绘制 {loc} 的置信区间时出错: {plot_ci_error}")
        ax.set_title(loc)
        ax.set_ylabel('比例 (%)')
        ax.legend(fontsize='small')
        ax.grid(True, linestyle=':', alpha=0.6)

    for j in range(i + 1, len(axes_flat)):
        axes_flat[j].set_visible(False)
    for row in range(n_rows):
        for col in range(cols_per_row):
            ax = axes[row, col]
            if ax.is_visible():
                if row == n_rows - 1:
                    ax.set_xlabel('年份')
                else:
                    ax.tick_params(labelbottom=False)

    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.show()

共 36 个地区/城市需要处理。
开始为每个地区/城市拟合 ARIMA 模型并预测...

--- 正在处理: 北京 (1/36) ---
为 '北京' 找到的最佳 ARIMA 模型阶数: (0, 2, 0)
模型详细参数:
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   12
Model:               SARIMAX(0, 2, 0)   Log Likelihood                 -33.683
Date:                Wed, 26 Mar 2025   AIC                             69.366
Time:                        12:36:12   BIC                             69.669
Sample:                    12-31-2010   HQIC                            69.034
                         - 12-31-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2        49.3411     26.923      1.833      0.067      -3.427     102.109
Ljung-Box (L1) (Q)

In [15]:
import pandas as pd
import numpy as np
import pmdarima as pm
import warnings
import matplotlib.pyplot as plt # 可选，用于绘图
import os
import sys # <--- 新增导入
import contextlib # <--- 新增导入

warnings.filterwarnings("ignore") # 忽略 ARIMA 拟合中可能出现的警告

# --- 新增：定义标准输出重定向的上下文管理器 ---
@contextlib.contextmanager
def redirect_stdout_to_file(filename, mode='w', encoding='utf-8'):
    """一个临时重定向 sys.stdout 到文件的上下文管理器。"""
    original_stdout = sys.stdout
    try:
        with open(filename, mode, encoding=encoding) as f:
            sys.stdout = f
            yield f # 可以选择性地 yield 文件对象，虽然这里没用到
    finally:
        sys.stdout = original_stdout # 保证无论如何都恢复 stdout
# --- 结束新增部分 ---

# 1. 准备数据 (保持不变)
data = {
    '年份': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
    # 第1批
    '北京': [105.77, 114.81, 123.23, 122.35, 108.59, 102.95, 102.90, 105.52, 110.17, 103.46, 95.26, 86.09],
    '天津': [91.30, 89.88, 94.47, 103.68, 109.48, 113.04, 117.02, 124.33, 130.42, 130.90, 131.46, 131.84],
    '河北': [81.48, 83.65, 82.64, 94.45, 101.51, 106.19, 111.22, 116.77, 117.40, 114.08, 113.52, 116.03],
    '山西': [106.53, 97.57, 86.21, 86.05, 84.72, 85.81, 86.35, 92.07, 93.91, 93.60, 93.59, 94.86],
    # 第2批
    '内蒙古': [72.29, 71.91, 70.61, 74.37, 78.32, 82.76, 89.48, 93.14, 94.75, 91.87, 90.72, 90.92],
    '辽宁': [103.02, 104.08, 95.14, 93.12, 90.92, 98.01, 105.61, 107.50, 110.64, 109.81, 111.02, 107.74],
    '吉林': [77.11, 73.96, 72.69, 76.36, 79.74, 88.03, 96.87, 107.23, 105.96, 99.26, 96.07, 97.28],
    '黑龙江': [88.53, 90.84, 74.73, 80.20, 86.93, 95.15, 101.55, 107.77, 105.56, 98.37, 93.57, 89.78],
    '上海': [195.87, 199.82, 193.81, 181.52, 172.33, 174.85, 164.24, 145.55, 124.86, 107.03, 98.66, 94.89],
    '江苏': [83.99, 81.35, 80.91, 85.81, 90.82, 96.50, 101.85, 108.33, 114.54, 118.61, 122.16, 123.76],
    '浙江': [95.03, 95.18, 96.93, 99.74, 98.58, 100.72, 102.85, 101.99, 98.31, 95.85, 97.84, 99.69],
    '安徽': [85.50, 73.37, 71.22, 73.42, 79.42, 87.93, 95.32, 98.07, 99.10, 98.24, 98.99, 97.66],
    '福建': [93.20, 96.78, 98.96, 92.71, 91.27, 91.61, 95.92, 100.46, 104.50, 109.12, 113.10, 114.58],
    '江西': [91.68, 90.74, 83.99, 86.68, 90.21, 94.98, 103.69, 115.24, 123.49, 122.90, 119.55, 114.30],
    # 第3批
    '山东': [121.32, 110.07, 105.51, 105.12, 103.49, 103.36, 103.89, 104.77, 107.04, 111.80, 120.15, 123.80],
    '河南': [119.02, 115.10, 98.15, 103.26, 107.33, 113.30, 120.59, 131.85, 140.76, 144.97, 149.03, 154.16],
    '湖北': [76.88, 65.12, 69.80, 75.08, 85.13, 96.42, 104.18, 110.41, 112.28, 112.53, 114.12, 115.98],
    '湖南': [87.93, 87.41, 91.02, 95.44, 96.64, 97.00, 98.33, 103.43, 107.88, 110.59, 113.10, 113.49],
    '广东': [132.39, 126.82, 118.09, 108.50, 101.99, 100.56, 103.51, 107.68, 111.10, 113.86, 117.76, 121.94],
    '广西': [87.69, 88.24, 90.82, 94.67, 98.77, 100.95, 103.36, 108.07, 111.08, 112.09, 112.72, 115.99],
    '海南': [90.19, 85.91, 85.10, 86.79, 87.93, 88.40, 91.60, 97.08, 101.23, 103.81, 106.05, 108.69],
    '重庆': [102.75, 100.41, 101.57, 106.55, 114.16, 121.60, 124.25, 125.97, 133.25, 137.74, 135.04, 127.64],
    '四川': [81.00, 79.10, 76.08, 79.77, 84.60, 93.11, 102.90, 113.31, 119.91, 122.47, 123.51, 123.87],
    '贵州': [89.62, 86.83, 86.88, 87.79, 88.32, 90.12, 93.71, 99.87, 106.60, 112.67, 119.83, 130.30],
    '云南': [80.81, 77.35, 75.27, 77.48, 78.58, 79.47, 82.45, 86.30, 91.38, 96.33, 101.33, 105.39],
    '西藏': [74.32, 70.98, 69.00, 69.93, 69.96, 76.07, 82.03, 86.08, 93.34, 96.17, 98.87, 105.35],
    '陕西': [96.26, 88.81, 86.26, 86.47, 88.89, 90.21, 90.20, 93.59, 98.10, 103.18, 106.50, 111.76],
    '甘肃': [95.20, 92.64, 85.75, 85.41, 85.01, 86.11, 87.14, 90.39, 91.84, 89.78, 89.34, 90.00],
    '青海': [69.02, 64.55, 65.40, 68.36, 70.75, 71.31, 73.23, 82.55, 86.22, 88.09, 88.71, 92.20],
    '宁夏': [99.90, 98.09, 97.06, 97.05, 97.08, 97.99, 99.57, 103.29, 105.57, 103.56, 103.08, 103.77],
    '新疆': [97.27, 92.13, 89.93, 91.16, 93.39, 94.14, 95.66, 97.02, 100.66, 104.38, 108.85, 112.66],
    '大连市': [127.53, 127.86, 125.98, 122.01, 123.20, 123.97, 130.33, 133.51, 141.03, 141.90, 138.07, 134.99],
    '宁波市': [132.53, 128.27, 128.12, 130.14, 139.07, 145.67, 149.69, 147.43, 153.23, 158.37, 161.93, 163.06],
    '厦门市': [112.40, 112.22, 114.87, 219.25, 227.30, 243.44, 254.68, 253.30, 250.15, 246.08, 237.01, 230.31],
    # 第4批 (本次新增)
    '青岛市': [111.81, 110.71, 113.06, 113.91, 114.40, 115.71, 118.40, 120.25, 121.36, 124.85, 130.45, 138.36],
    '深圳市': [597.57, 603.29, 578.65, 534.73, 482.72, 460.55, 474.07, 490.73, 484.91, 436.45, 402.13, 382.65]
}
df = pd.DataFrame(data)
df['年份'] = pd.to_datetime(df['年份'], format='%Y')
df.set_index('年份', inplace=True)
df.index = df.index.to_period('A')

# 2. 定义预测参数 (保持不变)
locations = list(data.keys())[1:]
forecast_horizon = 2035 - 2021

# --- 定义输出文件名 ---
output_log_filename = 'arima_processing_log.txt'
output_csv_filename = 'arima_forecast_2022_2035.csv'

# --- 在重定向前打印一条消息到控制台 ---
print(f"--- 开始处理，所有详细文本输出将写入文件: {output_log_filename} ---")
print(f"--- 最终预测表将导出到 CSV 文件: {output_csv_filename} ---")

# --- 使用上下文管理器重定向输出 ---
try:
    with redirect_stdout_to_file(output_log_filename, encoding='utf-8'):

        # 3. 循环为每个地区/城市进行 ARIMA 预测 (现在 print 会写入文件)
        forecast_results = {}
        fitted_models = {}

        print(f"共 {len(locations)} 个地区/城市需要处理。") # <-- 写入文件
        print("开始为每个地区/城市拟合 ARIMA 模型并预测...\n") # <-- 写入文件

        for i, loc in enumerate(locations):
            print(f"--- 正在处理: {loc} ({i+1}/{len(locations)}) ---") # <-- 写入文件
            series = df[loc]
            try:
                auto_model = pm.auto_arima(series, start_p=0, start_q=0, max_p=3, max_q=3, m=1, d=None, test='adf',
                                           seasonal=False, stepwise=True, suppress_warnings=True, error_action='ignore', trace=False)
                if auto_model is None or not hasattr(auto_model, 'order'):
                     raise ValueError("auto_arima未能找到合适的模型。")
                print(f"为 '{loc}' 找到的最佳 ARIMA 模型阶数: {auto_model.order}") # <-- 写入文件
                print("模型详细参数:") # <-- 写入文件
                print(auto_model.summary()) # <-- 写入文件 (summary 的输出也会被捕获)
                print("-" * 60) # <-- 写入文件
                fitted_models[loc] = auto_model
                forecast, conf_int = auto_model.predict(n_periods=forecast_horizon, return_conf_int=True)
                forecast_results[loc] = forecast
            except Exception as e:
                print(f"!!! 无法为 '{loc}' 拟合或预测 ARIMA 模型: {e}") # <-- 写入文件
                print("   可能是因为数据序列过短、模式不明显、数值过大/波动剧烈或auto_arima搜索失败。") # <-- 写入文件
                forecast_results[loc] = [np.nan] * forecast_horizon
                fitted_models[loc] = None
            finally:
                print("\n") # <-- 写入文件


        # 4. 整理并展示预测结果 (现在 print 会写入文件)
        forecast_index = pd.period_range(start=str(df.index[-1].year + 1), periods=forecast_horizon, freq='A')
        forecast_df = pd.DataFrame(forecast_results, index=forecast_index)
        forecast_df_formatted = forecast_df.round(2).astype(str) + '%'
        forecast_df_formatted = forecast_df_formatted.replace('nan%', 'N/A')

        print("\n\n--- ARIMA 模型预测结果 (2022-2035) ---") # <-- 写入文件

        # 为了确保 DataFrame 完整打印到文件，使用 to_string()
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 2000) # 设置足够宽的宽度
        print(forecast_df_formatted.to_string()) # <-- 将完整表格写入文件
        pd.reset_option('display.max_rows')
        pd.reset_option('display.max_columns')
        pd.reset_option('display.width')

        # 导出到 CSV (打印的确认信息也会写入 TXT 文件)
        try:
            # 使用 utf-8-sig 编码确保 Excel 正确识别中文和特殊字符
            forecast_df_formatted.to_csv(output_csv_filename, encoding='utf-8-sig')
            print(f"\n预测结果已成功导出到 CSV 文件: {os.path.abspath(output_csv_filename)}") # <-- 写入文件
        except Exception as e:
            print(f"\n!!! 导出 CSV 文件时出错: {e}") # <-- 写入文件

        # 打印绘图准备信息 (也会写入 TXT 文件)
        plot_results = True # 设置为 False 则不绘图
        if plot_results:
            print("\n--- 正在准备绘制结果图表 (图表将单独显示)... ---") # <-- 写入文件

    # --- 重定向结束，后续 print 会回到控制台 ---
    print(f"--- 处理完成，详细文本输出已写入: {os.path.abspath(output_log_filename)} ---")

except Exception as global_error:
    # 捕获在重定向期间可能发生的其他未预料错误
    print(f"!!! 脚本执行过程中发生严重错误: {global_error} !!!")
    # 尝试将错误信息追加到日志文件
    try:
        with open(output_log_filename, 'a', encoding='utf-8') as f_err:
             f_err.write(f"\n\n!!! 脚本执行过程中发生严重错误: {global_error} !!!\n")
    except:
        pass # 忽略记录错误时发生的错误



--- 开始处理，所有详细文本输出将写入文件: arima_processing_log.txt ---
--- 最终预测表将导出到 CSV 文件: arima_forecast_2022_2035.csv ---
--- 处理完成，详细文本输出已写入: /Users/annfengdeye/Desktop/3.26下午2点/arima_processing_log.txt ---


In [29]:
import pandas as pd
import numpy as np
import pmdarima as pm
import warnings
import matplotlib.pyplot as plt # 可选，用于绘图
import os # 导入 os 模块用于文件路径操作

warnings.filterwarnings("ignore") # 忽略 ARIMA 拟合中可能出现的警告

# ... (代码的第 1, 2, 3 部分保持不变) ...

# 1. 准备数据 (包含所有批次的数据)
# --- 更新 data 字典 ---
data = {
    '年份': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
'北京': [91.12, 85.19, 88.3, 79.79, 88.03, 87.57, 86.32, 80.39, 81.09, 90.84, 97.4, 98.98],
'天津': [87.19, 83.24, 82.62, 75.72, 77.12, 78.08, 74.91, 78.94, 88.76, 98.67, 104.01, 105.82],
'河北': [67.42, 71.11, 78.0, 76.29, 82.61, 83.1, 83.82, 85.96, 87.67, 90.33, 90.7, 87.23],
'山西': [70.3, 79.35, 84.7, 84.01, 87.18, 87.52, 89.91, 87.07, 89.85, 90.67, 92.22, 90.91],
'内蒙古': [61.09, 66.99, 77.32, 73.72, 87.15, 90.38, 95.29, 93.92, 95.21, 95.72, 93.47, 94.49],
'辽宁': [97.83, 98.17, 101.2, 102.5, 113.75, 116.98, 123.8, 114.38, 114.07, 104.41, 107.19, 97.38],
'吉林': [73.57, 71.09, 76.84, 74.55, 82.06, 90.09, 95.57, 88.38, 86.87, 86.1, 88.15, 82.61],
'黑龙江': [62.98, 64.98, 60.95, 63.5, 64.3, 65.39, 70.39, 68.2, 71.22, 71.79, 76.97, 75.12],
'上海': [88.07, 86.96, 90.19, 94.96, 100.9, 97.92, 94.0, 93.72, 95.17, 93.8, 92.89, 92.84],
'江苏': [100.68, 100.94, 105.62, 102.89, 110.02, 112.31, 113.17, 110.89, 109.36, 107.53, 105.25, 98.99],
'浙江': [109.14, 109.62, 106.31, 107.26, 107.54, 105.45, 104.36, 99.63, 96.46, 93.39, 90.4, 87.57],
'安徽': [50.3, 67.39, 73.72, 72.61, 81.41, 84.69, 87.43, 89.43, 90.99, 91.47, 89.23, 86.52],
'福建': [100.26, 104.65, 108.62, 107.09, 111.93, 111.86, 111.47, 106.54, 101.54, 95.54, 92.18, 86.85],
'江西': [80.38, 83.45, 87.03, 85.44, 91.3, 88.12, 89.9, 90.61, 93.05, 94.32, 88.73, 83.2],
'山东': [74.28, 77.39, 79.18, 77.45, 79.33, 79.85, 77.24, 81.64, 81.68, 81.74, 78.71, 82.5],
'河南': [87.77, 99.98, 115.06, 107.07, 119.88, 122.07, 122.83, 122.35, 116.76, 112.9, 106.46, 102.8],
'湖北': [85.31, 85.88, 94.65, 86.89, 91.42, 91.93, 91.74, 88.77, 89.38, 87.1, 84.88, 81.88],
'湖南': [71.94, 75.91, 84.22, 76.05, 83.4, 85.13, 85.76, 84.05, 84.76, 86.25, 87.17, 87.01],
'广东': [84.53, 88.96, 101.79, 94.29, 107.95, 111.42, 114.46, 113.68, 114.44, 124.02, 106.4, 100.44],
'广西': [70.74, 81.48, 97.58, 91.12, 105.04, 106.43, 107.61, 109.37, 107.74, 110.82, 108.77, 103.37],
'海南': [56.63, 72.31, 83.41, 78.5, 82.93, 88.69, 91.6, 92.81, 91.92, 96.17, 93.31, 91.59],
'重庆': [100.5, 104.77, 106.08, 107.18, 107.25, 104.06, 102.29, 100.02, 98.47, 97.12, 94.25, 91.97],
'四川': [93.14, 97.87, 108.15, 103.63, 108.72, 110.33, 108.95, 106.4, 105.63, 102.65, 97.95, 93.17],
'贵州': [58.44, 67.43, 82.82, 74.54, 91.0, 101.92, 108.53, 109.0, 107.27, 107.13, 107.85, 105.53],
'云南': [59.96, 63.48, 70.71, 67.51, 73.29, 74.6, 79.43, 81.12, 83.86, 91.63, 95.23, 97.31],
'西藏': [28.44, 43.03, 57.81, 52.56, 62.49, 67.38, 72.41, 82.02, 92.27, 95.93, 97.43, 96.69],
'陕西': [90.87, 103.25, 117.3, 113.79, 125.79, 127.88, 127.45, 122.74, 118.83, 117.06, 110.27, 120.78],
'甘肃': [43.64, 47.83, 62.2, 56.65, 74.46, 96.29, 100.26, 101.62, 100.31, 102.8, 105.6, 107.44],
'青海': [53.5, 61.94, 70.92, 67.53, 74.73, 81.45, 84.54, 86.52, 85.28, 89.32, 89.83, 87.59],
'宁夏': [53.5, 58.52, 66.81, 63.08, 72.88, 78.71, 88.22, 92.48, 93.65, 94.41, 93.85, 89.92],
'新疆': [63.46, 66.4, 71.58, 69.46, 79.29, 89.55, 136.9, 142.45, 136.85, 118.59, 103.63, 92.53],
'大连市': [109.4, 109.35, 109.23, 108.73, 108.15, 102.72, 99.2, 101.14, 97.27, 96.98, 91.14, 99.85],
'宁波市': [192.29, 197.39, 198.22, 196.96, 232.02, 279.7, 267.52, 217.48, 186.66, 183.85, 179.88, 179.05],
'厦门市': [151.89, 162.83, 172.58, 183.14, 182.79, 177.07, 170.93, 167.4, 155.47, 153.11, 146.2, 147.66],
'青岛市': [89.96, 102.93, 112.71, 112.65, 108.83, 108.34, 103.45, 106.53, 104.86, 103.89, 113.05, 121.38],
'深圳市': [285.58, 277.18, 284.98, 316.08, 303.68, 286.36, 263.83, 260.86, 246.73, 217.35, 191.39, 184.1]
}
df = pd.DataFrame(data)
df['年份'] = pd.to_datetime(df['年份'], format='%Y')
df.set_index('年份', inplace=True)
df.index = df.index.to_period('A')

# 2. 定义预测参数
locations = list(data.keys())[1:]
forecast_horizon = 2035 - 2021

# 3. 循环为每个地区/城市进行 ARIMA 预测
forecast_results = {}
fitted_models = {}

print(f"共 {len(locations)} 个地区/城市需要处理。")
print("开始为每个地区/城市拟合 ARIMA 模型并预测...\n")

# ... (ARIMA 拟合和预测的循环代码不变) ...
for i, loc in enumerate(locations):
    print(f"--- 正在处理: {loc} ({i+1}/{len(locations)}) ---")
    series = df[loc]
    try:
        auto_model = pm.auto_arima(series, start_p=0, start_q=0, max_p=3, max_q=3, m=1, d=None, test='adf',
                                   seasonal=False, stepwise=True, suppress_warnings=True, error_action='ignore', trace=False)
        if auto_model is None or not hasattr(auto_model, 'order'):
             raise ValueError("auto_arima未能找到合适的模型。")
        print(f"为 '{loc}' 找到的最佳 ARIMA 模型阶数: {auto_model.order}")
        print("模型详细参数:")
        print(auto_model.summary())
        print("-" * 60)
        fitted_models[loc] = auto_model
        forecast, conf_int = auto_model.predict(n_periods=forecast_horizon, return_conf_int=True)
        forecast_results[loc] = forecast
    except Exception as e:
        print(f"!!! 无法为 '{loc}' 拟合或预测 ARIMA 模型: {e}")
        print("   可能是因为数据序列过短、模式不明显、数值过大/波动剧烈或auto_arima搜索失败。")
        forecast_results[loc] = [np.nan] * forecast_horizon
        fitted_models[loc] = None
    finally:
        print("\n")


# 4. 整理并展示预测结果
forecast_index = pd.period_range(start=str(df.index[-1].year + 1), periods=forecast_horizon, freq='A')
forecast_df = pd.DataFrame(forecast_results, index=forecast_index)
forecast_df_formatted = forecast_df.round(2).astype(str) + '%'
forecast_df_formatted = forecast_df_formatted.replace('nan%', 'N/A')

print("\n\n--- ARIMA 模型预测结果 (2022-2035) ---")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(forecast_df_formatted)
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

# --- 新增：导出到 CSV ---
output_filename = 'arima_forecast_2022_2035.csv'
try:
    # 使用 utf-8-sig 编码确保 Excel 正确识别中文和特殊字符
    forecast_df_formatted.to_csv(output_filename, encoding='utf-8-sig')
    print(f"\n预测结果已成功导出到文件: {os.path.abspath(output_filename)}") # 打印完整路径
except Exception as e:
    print(f"\n!!! 导出 CSV 文件时出错: {e}")
# --- 结束新增部分 ---


# 5. 可选：绘制历史数据和预测结果
plot_results = False # 设置为 False 则不绘图

if plot_results:
    print("\n--- 正在绘制结果图表 ---")
    # ... (绘图代码不变) ...
    n_locations = len(locations)
    cols_per_row = 3
    n_rows = (n_locations + cols_per_row - 1) // cols_per_row
    fig_width = 15
    fig_height_per_row = 4
    fig_height = n_rows * fig_height_per_row

    fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(fig_width, fig_height), sharex=True, squeeze=False)
    fig.suptitle('各地区/城市比例历史数据与 ARIMA 预测 (2010-2035)', fontsize=16, y=0.99)

    try:
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
    except Exception as font_error:
        print(f"\n注意: 设置中文字体失败: {font_error}")
        print("图表标题和标签可能显示乱码。")

    axes_flat = axes.flatten()

    for i, loc in enumerate(locations):
        ax = axes_flat[i]
        if loc in df.columns:
             df[loc].plot(ax=ax, label='历史数据', marker='o', linestyle='-')
        if loc in forecast_df.columns and forecast_df[loc].notna().any():
            forecast_df[loc].plot(ax=ax, label='ARIMA 预测', linestyle='--', marker='x')
            if fitted_models.get(loc) is not None:
                try:
                    _, conf_int_plot = fitted_models[loc].predict(n_periods=forecast_horizon, return_conf_int=True)
                    conf_int_df = pd.DataFrame(conf_int_plot, index=forecast_index, columns=['lower', 'upper'])
                    conf_int_df['lower'] = conf_int_df['lower'].clip(lower=0)
                    ax.fill_between(conf_int_df.index.to_timestamp(),
                                    conf_int_df['lower'],
                                    conf_int_df['upper'],
                                    color='gray', alpha=0.2, label='95% 置信区间 (≥0)')
                except Exception as plot_ci_error:
                     print(f"绘制 {loc} 的置信区间时出错: {plot_ci_error}")
        ax.set_title(loc)
        ax.set_ylabel('比例 (%)')
        ax.legend(fontsize='small')
        ax.grid(True, linestyle=':', alpha=0.6)

    for j in range(i + 1, len(axes_flat)):
        axes_flat[j].set_visible(False)
    for row in range(n_rows):
        for col in range(cols_per_row):
            ax = axes[row, col]
            if ax.is_visible():
                if row == n_rows - 1:
                    ax.set_xlabel('年份')
                else:
                    ax.tick_params(labelbottom=False)

    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.show()

共 36 个地区/城市需要处理。
开始为每个地区/城市拟合 ARIMA 模型并预测...

--- 正在处理: 北京 (1/36) ---
为 '北京' 找到的最佳 ARIMA 模型阶数: (0, 2, 1)
模型详细参数:
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   12
Model:               SARIMAX(0, 2, 1)   Log Likelihood                 -33.220
Date:                Wed, 26 Mar 2025   AIC                             70.440
Time:                        12:42:36   BIC                             71.046
Sample:                    12-31-2010   HQIC                            69.777
                         - 12-31-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.8209      0.700     -1.173      0.241      -2.193       0.551
sigma2        40.2

In [34]:
import pandas as pd
import numpy as np
import pmdarima as pm
import warnings
import matplotlib.pyplot as plt # 可选，用于绘图
import os
import sys # <--- 新增导入
import contextlib # <--- 新增导入

warnings.filterwarnings("ignore") # 忽略 ARIMA 拟合中可能出现的警告

# --- 新增：定义标准输出重定向的上下文管理器 ---
@contextlib.contextmanager
def redirect_stdout_to_file(filename, mode='w', encoding='utf-8'):
    """一个临时重定向 sys.stdout 到文件的上下文管理器。"""
    original_stdout = sys.stdout
    try:
        with open(filename, mode, encoding=encoding) as f:
            sys.stdout = f
            yield f # 可以选择性地 yield 文件对象，虽然这里没用到
    finally:
        sys.stdout = original_stdout # 保证无论如何都恢复 stdout
# --- 结束新增部分 ---

# 1. 准备数据 (保持不变)
data = {
    '年份': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
'北京': [91.12, 85.19, 88.3, 79.79, 88.03, 87.57, 86.32, 80.39, 81.09, 90.84, 97.4, 98.98],
'天津': [87.19, 83.24, 82.62, 75.72, 77.12, 78.08, 74.91, 78.94, 88.76, 98.67, 104.01, 105.82],
'河北': [67.42, 71.11, 78.0, 76.29, 82.61, 83.1, 83.82, 85.96, 87.67, 90.33, 90.7, 87.23],
'山西': [70.3, 79.35, 84.7, 84.01, 87.18, 87.52, 89.91, 87.07, 89.85, 90.67, 92.22, 90.91],
'内蒙古': [61.09, 66.99, 77.32, 73.72, 87.15, 90.38, 95.29, 93.92, 95.21, 95.72, 93.47, 94.49],
'辽宁': [97.83, 98.17, 101.2, 102.5, 113.75, 116.98, 123.8, 114.38, 114.07, 104.41, 107.19, 97.38],
'吉林': [73.57, 71.09, 76.84, 74.55, 82.06, 90.09, 95.57, 88.38, 86.87, 86.1, 88.15, 82.61],
'黑龙江': [62.98, 64.98, 60.95, 63.5, 64.3, 65.39, 70.39, 68.2, 71.22, 71.79, 76.97, 75.12],
'上海': [88.07, 86.96, 90.19, 94.96, 100.9, 97.92, 94.0, 93.72, 95.17, 93.8, 92.89, 92.84],
'江苏': [100.68, 100.94, 105.62, 102.89, 110.02, 112.31, 113.17, 110.89, 109.36, 107.53, 105.25, 98.99],
'浙江': [109.14, 109.62, 106.31, 107.26, 107.54, 105.45, 104.36, 99.63, 96.46, 93.39, 90.4, 87.57],
'安徽': [50.3, 67.39, 73.72, 72.61, 81.41, 84.69, 87.43, 89.43, 90.99, 91.47, 89.23, 86.52],
'福建': [100.26, 104.65, 108.62, 107.09, 111.93, 111.86, 111.47, 106.54, 101.54, 95.54, 92.18, 86.85],
'江西': [80.38, 83.45, 87.03, 85.44, 91.3, 88.12, 89.9, 90.61, 93.05, 94.32, 88.73, 83.2],
'山东': [74.28, 77.39, 79.18, 77.45, 79.33, 79.85, 77.24, 81.64, 81.68, 81.74, 78.71, 82.5],
'河南': [87.77, 99.98, 115.06, 107.07, 119.88, 122.07, 122.83, 122.35, 116.76, 112.9, 106.46, 102.8],
'湖北': [85.31, 85.88, 94.65, 86.89, 91.42, 91.93, 91.74, 88.77, 89.38, 87.1, 84.88, 81.88],
'湖南': [71.94, 75.91, 84.22, 76.05, 83.4, 85.13, 85.76, 84.05, 84.76, 86.25, 87.17, 87.01],
'广东': [84.53, 88.96, 101.79, 94.29, 107.95, 111.42, 114.46, 113.68, 114.44, 124.02, 106.4, 100.44],
'广西': [70.74, 81.48, 97.58, 91.12, 105.04, 106.43, 107.61, 109.37, 107.74, 110.82, 108.77, 103.37],
'海南': [56.63, 72.31, 83.41, 78.5, 82.93, 88.69, 91.6, 92.81, 91.92, 96.17, 93.31, 91.59],
'重庆': [100.5, 104.77, 106.08, 107.18, 107.25, 104.06, 102.29, 100.02, 98.47, 97.12, 94.25, 91.97],
'四川': [93.14, 97.87, 108.15, 103.63, 108.72, 110.33, 108.95, 106.4, 105.63, 102.65, 97.95, 93.17],
'贵州': [58.44, 67.43, 82.82, 74.54, 91.0, 101.92, 108.53, 109.0, 107.27, 107.13, 107.85, 105.53],
'云南': [59.96, 63.48, 70.71, 67.51, 73.29, 74.6, 79.43, 81.12, 83.86, 91.63, 95.23, 97.31],
'西藏': [28.44, 43.03, 57.81, 52.56, 62.49, 67.38, 72.41, 82.02, 92.27, 95.93, 97.43, 96.69],
'陕西': [90.87, 103.25, 117.3, 113.79, 125.79, 127.88, 127.45, 122.74, 118.83, 117.06, 110.27, 120.78],
'甘肃': [43.64, 47.83, 62.2, 56.65, 74.46, 96.29, 100.26, 101.62, 100.31, 102.8, 105.6, 107.44],
'青海': [53.5, 61.94, 70.92, 67.53, 74.73, 81.45, 84.54, 86.52, 85.28, 89.32, 89.83, 87.59],
'宁夏': [53.5, 58.52, 66.81, 63.08, 72.88, 78.71, 88.22, 92.48, 93.65, 94.41, 93.85, 89.92],
'新疆': [63.46, 66.4, 71.58, 69.46, 79.29, 89.55, 136.9, 142.45, 136.85, 118.59, 103.63, 92.53],
'大连市': [109.4, 109.35, 109.23, 108.73, 108.15, 102.72, 99.2, 101.14, 97.27, 96.98, 91.14, 99.85],
'宁波市': [192.29, 197.39, 198.22, 196.96, 232.02, 279.7, 267.52, 217.48, 186.66, 183.85, 179.88, 179.05],
'厦门市': [151.89, 162.83, 172.58, 183.14, 182.79, 177.07, 170.93, 167.4, 155.47, 153.11, 146.2, 147.66],
'青岛市': [89.96, 102.93, 112.71, 112.65, 108.83, 108.34, 103.45, 106.53, 104.86, 103.89, 113.05, 121.38],
'深圳市': [285.58, 277.18, 284.98, 316.08, 303.68, 286.36, 263.83, 260.86, 246.73, 217.35, 191.39, 184.1]
}
df = pd.DataFrame(data)
df['年份'] = pd.to_datetime(df['年份'], format='%Y')
df.set_index('年份', inplace=True)
df.index = df.index.to_period('A')

# 2. 定义预测参数 (保持不变)
locations = list(data.keys())[1:]
forecast_horizon = 2035 - 2021

# --- 定义输出文件名 ---
output_log_filename = 'arima_processing_log.txt'
output_csv_filename = 'arima_forecast_2022_2035.csv'

# --- 在重定向前打印一条消息到控制台 ---
print(f"--- 开始处理，所有详细文本输出将写入文件: {output_log_filename} ---")
print(f"--- 最终预测表将导出到 CSV 文件: {output_csv_filename} ---")

# --- 使用上下文管理器重定向输出 ---
try:
    with redirect_stdout_to_file(output_log_filename, encoding='utf-8'):

        # 3. 循环为每个地区/城市进行 ARIMA 预测 (现在 print 会写入文件)
        forecast_results = {}
        fitted_models = {}

        print(f"共 {len(locations)} 个地区/城市需要处理。") # <-- 写入文件
        print("开始为每个地区/城市拟合 ARIMA 模型并预测...\n") # <-- 写入文件

        for i, loc in enumerate(locations):
            print(f"--- 正在处理: {loc} ({i+1}/{len(locations)}) ---") # <-- 写入文件
            series = df[loc]
            try:
                auto_model = pm.auto_arima(series, start_p=0, start_q=0, max_p=3, max_q=3, m=1, d=None, test='adf',
                                           seasonal=False, stepwise=True, suppress_warnings=True, error_action='ignore', trace=False)
                if auto_model is None or not hasattr(auto_model, 'order'):
                     raise ValueError("auto_arima未能找到合适的模型。")
                print(f"为 '{loc}' 找到的最佳 ARIMA 模型阶数: {auto_model.order}") # <-- 写入文件
                print("模型详细参数:") # <-- 写入文件
                print(auto_model.summary()) # <-- 写入文件 (summary 的输出也会被捕获)
                print("-" * 60) # <-- 写入文件
                fitted_models[loc] = auto_model
                forecast, conf_int = auto_model.predict(n_periods=forecast_horizon, return_conf_int=True)
                forecast_results[loc] = forecast
            except Exception as e:
                print(f"!!! 无法为 '{loc}' 拟合或预测 ARIMA 模型: {e}") # <-- 写入文件
                print("   可能是因为数据序列过短、模式不明显、数值过大/波动剧烈或auto_arima搜索失败。") # <-- 写入文件
                forecast_results[loc] = [np.nan] * forecast_horizon
                fitted_models[loc] = None
            finally:
                print("\n") # <-- 写入文件


        # 4. 整理并展示预测结果 (现在 print 会写入文件)
        forecast_index = pd.period_range(start=str(df.index[-1].year + 1), periods=forecast_horizon, freq='A')
        forecast_df = pd.DataFrame(forecast_results, index=forecast_index)
        forecast_df_formatted = forecast_df.round(2).astype(str) + '%'
        forecast_df_formatted = forecast_df_formatted.replace('nan%', 'N/A')

        print("\n\n--- ARIMA 模型预测结果 (2022-2035) ---") # <-- 写入文件

        # 为了确保 DataFrame 完整打印到文件，使用 to_string()
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 2000) # 设置足够宽的宽度
        print(forecast_df_formatted.to_string()) # <-- 将完整表格写入文件
        pd.reset_option('display.max_rows')
        pd.reset_option('display.max_columns')
        pd.reset_option('display.width')

        # 导出到 CSV (打印的确认信息也会写入 TXT 文件)
        try:
            # 使用 utf-8-sig 编码确保 Excel 正确识别中文和特殊字符
            forecast_df_formatted.to_csv(output_csv_filename, encoding='utf-8-sig')
            print(f"\n预测结果已成功导出到 CSV 文件: {os.path.abspath(output_csv_filename)}") # <-- 写入文件
        except Exception as e:
            print(f"\n!!! 导出 CSV 文件时出错: {e}") # <-- 写入文件

        # 打印绘图准备信息 (也会写入 TXT 文件)
        plot_results = True # 设置为 False 则不绘图
        if plot_results:
            print("\n--- 正在准备绘制结果图表 (图表将单独显示)... ---") # <-- 写入文件

    # --- 重定向结束，后续 print 会回到控制台 ---
    print(f"--- 处理完成，详细文本输出已写入: {os.path.abspath(output_log_filename)} ---")

except Exception as global_error:
    # 捕获在重定向期间可能发生的其他未预料错误
    print(f"!!! 脚本执行过程中发生严重错误: {global_error} !!!")
    # 尝试将错误信息追加到日志文件
    try:
        with open(output_log_filename, 'a', encoding='utf-8') as f_err:
             f_err.write(f"\n\n!!! 脚本执行过程中发生严重错误: {global_error} !!!\n")
    except:
        pass # 忽略记录错误时发生的错误



--- 开始处理，所有详细文本输出将写入文件: arima_processing_log.txt ---
--- 最终预测表将导出到 CSV 文件: arima_forecast_2022_2035.csv ---
--- 处理完成，详细文本输出已写入: /Users/annfengdeye/Desktop/3.26下午2点/arima_processing_log.txt ---


In [37]:
import pandas as pd
import numpy as np
import pmdarima as pm
import warnings
import matplotlib.pyplot as plt # 可选，用于绘图
import os # 导入 os 模块用于文件路径操作

warnings.filterwarnings("ignore") # 忽略 ARIMA 拟合中可能出现的警告

# ... (代码的第 1, 2, 3 部分保持不变) ...

# 1. 准备数据 (包含所有批次的数据)
# --- 更新 data 字典 ---
data = {
'年份': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
'北京': [105.77, 114.81, 123.23, 122.35, 108.59, 102.95, 102.9, 105.52, 110.17, 103.46, 95.26, 86.09],
'天津': [91.3, 89.88, 94.47, 103.68, 109.48, 113.04, 117.02, 124.33, 130.42, 130.9, 131.46, 131.84],
'河北': [81.48, 83.65, 82.64, 94.45, 101.51, 106.19, 111.22, 116.77, 117.4, 114.08, 113.52, 116.03],
'山西': [106.53, 97.57, 86.21, 86.05, 84.72, 85.81, 86.35, 92.07, 93.91, 93.6, 93.59, 94.86],
'内蒙古': [72.29, 71.91, 70.61, 74.37, 78.32, 82.76, 89.48, 93.14, 94.75, 91.87, 90.72, 90.92],
'辽宁': [103.02, 104.08, 95.14, 93.12, 90.92, 98.01, 105.61, 107.5, 110.64, 109.81, 111.02, 107.74],
'吉林': [77.11, 73.96, 72.69, 76.36, 79.74, 88.03, 96.87, 107.23, 105.96, 99.26, 96.07, 97.28],
'黑龙江': [88.53, 90.84, 74.73, 80.2, 86.93, 95.15, 101.55, 107.77, 105.56, 98.37, 93.57, 89.78],
'上海': [195.87, 199.82, 193.81, 181.52, 172.33, 174.85, 164.24, 145.55, 124.86, 107.03, 98.66, 94.89],
'江苏': [83.99, 81.35, 80.91, 85.81, 90.82, 96.5, 101.85, 108.33, 114.54, 118.61, 122.16, 123.76],
'浙江': [95.03, 95.18, 96.93, 99.74, 98.58, 100.72, 102.85, 101.99, 98.31, 95.85, 97.84, 99.69],
'安徽': [85.5, 73.37, 71.22, 73.42, 79.42, 87.93, 95.32, 98.07, 99.1, 98.24, 98.99, 97.66],
'福建': [93.2, 96.78, 98.96, 92.71, 91.27, 91.61, 95.92, 100.46, 104.5, 109.12, 113.1, 114.58],
'江西': [91.68, 90.74, 83.99, 86.68, 90.21, 94.98, 103.69, 115.24, 123.49, 122.9, 119.55, 114.3],
'山东': [121.32, 110.07, 105.51, 105.12, 103.49, 103.36, 103.89, 104.77, 107.04, 111.8, 120.15, 123.8],
'河南': [119.02, 115.1, 98.15, 103.26, 107.33, 113.3, 120.59, 131.85, 140.76, 144.97, 149.03, 154.16],
'湖北': [76.88, 65.12, 69.8, 75.08, 85.13, 96.42, 104.18, 110.41, 112.28, 112.53, 114.12, 115.98],
'湖南': [87.93, 87.41, 91.02, 95.44, 96.64, 97.0, 98.33, 103.43, 107.88, 110.59, 113.1, 113.49],
'广东': [132.39, 126.82, 118.09, 108.5, 101.99, 100.56, 103.51, 107.68, 111.1, 113.86, 117.76, 121.94],
'广西': [87.69, 88.24, 90.82, 94.67, 98.77, 100.95, 103.36, 108.07, 111.08, 112.09, 112.72, 115.99],
'海南': [90.19, 85.91, 85.1, 86.79, 87.93, 88.4, 91.6, 97.08, 101.23, 103.81, 106.05, 108.69],
'重庆': [102.75, 100.41, 101.57, 106.55, 114.16, 121.6, 124.25, 125.97, 133.25, 137.74, 135.04, 127.64],
'四川': [81.0, 79.1, 76.08, 79.77, 84.6, 93.11, 102.9, 113.31, 119.91, 122.47, 123.51, 123.87],
'贵州': [89.62, 86.83, 86.88, 87.79, 88.32, 90.12, 93.71, 99.87, 106.6, 112.67, 119.83, 130.3],
'云南': [80.81, 77.35, 75.27, 77.48, 78.58, 79.47, 82.45, 86.3, 91.38, 96.33, 101.33, 105.39],
'西藏': [74.32, 70.98, 69.0, 69.93, 69.96, 76.07, 82.03, 86.08, 93.34, 96.17, 98.87, 105.35],
'陕西': [96.26, 88.81, 86.26, 86.47, 88.89, 90.21, 90.2, 93.59, 98.1, 103.18, 106.5, 111.76],
'甘肃': [95.2, 92.64, 85.75, 85.41, 85.01, 86.11, 87.14, 90.39, 91.84, 89.78, 89.34, 90.0],
'青海': [69.02, 64.55, 65.4, 68.36, 70.75, 71.31, 73.23, 82.55, 86.22, 88.09, 88.71, 92.2],
'宁夏': [99.9, 98.09, 97.06, 97.05, 97.08, 97.99, 99.57, 103.29, 105.57, 103.56, 103.08, 103.77],
'新疆': [97.27, 92.13, 89.93, 91.16, 93.39, 94.14, 95.66, 97.02, 100.66, 104.38, 108.85, 112.66],
'大连市': [127.53, 127.86, 125.98, 122.01, 123.2, 123.97, 130.33, 133.51, 141.03, 141.9, 138.07, 134.99],
'宁波市': [132.53, 128.27, 128.12, 130.14, 139.07, 145.67, 149.69, 147.43, 153.23, 158.37, 161.93, 163.06],
'厦门市': [112.4, 112.22, 114.87, 219.25, 227.3, 243.44, 254.68, 253.3, 250.15, 246.08, 237.01, 230.31],
'青岛市': [111.81, 110.71, 113.06, 113.91, 114.4, 115.71, 118.4, 120.25, 121.36, 124.85, 130.45, 138.36],
'深圳市': [597.57, 603.29, 578.65, 534.73, 482.72, 460.55, 474.07, 490.73, 484.91, 436.45, 402.13, 382.65]
}
df = pd.DataFrame(data)
df['年份'] = pd.to_datetime(df['年份'], format='%Y')
df.set_index('年份', inplace=True)
df.index = df.index.to_period('A')

# 2. 定义预测参数
locations = list(data.keys())[1:]
forecast_horizon = 2035 - 2021

# 3. 循环为每个地区/城市进行 ARIMA 预测
forecast_results = {}
fitted_models = {}

print(f"共 {len(locations)} 个地区/城市需要处理。")
print("开始为每个地区/城市拟合 ARIMA 模型并预测...\n")

# ... (ARIMA 拟合和预测的循环代码不变) ...
for i, loc in enumerate(locations):
    print(f"--- 正在处理: {loc} ({i+1}/{len(locations)}) ---")
    series = df[loc]
    try:
        auto_model = pm.auto_arima(series, start_p=0, start_q=0, max_p=3, max_q=3, m=1, d=None, test='adf',
                                   seasonal=False, stepwise=True, suppress_warnings=True, error_action='ignore', trace=False)
        if auto_model is None or not hasattr(auto_model, 'order'):
             raise ValueError("auto_arima未能找到合适的模型。")
        print(f"为 '{loc}' 找到的最佳 ARIMA 模型阶数: {auto_model.order}")
        print("模型详细参数:")
        print(auto_model.summary())
        print("-" * 60)
        fitted_models[loc] = auto_model
        forecast, conf_int = auto_model.predict(n_periods=forecast_horizon, return_conf_int=True)
        forecast_results[loc] = forecast
    except Exception as e:
        print(f"!!! 无法为 '{loc}' 拟合或预测 ARIMA 模型: {e}")
        print("   可能是因为数据序列过短、模式不明显、数值过大/波动剧烈或auto_arima搜索失败。")
        forecast_results[loc] = [np.nan] * forecast_horizon
        fitted_models[loc] = None
    finally:
        print("\n")


# 4. 整理并展示预测结果
forecast_index = pd.period_range(start=str(df.index[-1].year + 1), periods=forecast_horizon, freq='A')
forecast_df = pd.DataFrame(forecast_results, index=forecast_index)
forecast_df_formatted = forecast_df.round(2).astype(str) + '%'
forecast_df_formatted = forecast_df_formatted.replace('nan%', 'N/A')

print("\n\n--- ARIMA 模型预测结果 (2022-2035) ---")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(forecast_df_formatted)
pd.reset_option('display.max_columns')
pd.reset_option('display.width')

# --- 新增：导出到 CSV ---
output_filename = 'arima_forecast_2022_2035.csv'
try:
    # 使用 utf-8-sig 编码确保 Excel 正确识别中文和特殊字符
    forecast_df_formatted.to_csv(output_filename, encoding='utf-8-sig')
    print(f"\n预测结果已成功导出到文件: {os.path.abspath(output_filename)}") # 打印完整路径
except Exception as e:
    print(f"\n!!! 导出 CSV 文件时出错: {e}")
# --- 结束新增部分 ---


# 5. 可选：绘制历史数据和预测结果
plot_results = False # 设置为 False 则不绘图

if plot_results:
    print("\n--- 正在绘制结果图表 ---")
    # ... (绘图代码不变) ...
    n_locations = len(locations)
    cols_per_row = 3
    n_rows = (n_locations + cols_per_row - 1) // cols_per_row
    fig_width = 15
    fig_height_per_row = 4
    fig_height = n_rows * fig_height_per_row

    fig, axes = plt.subplots(n_rows, cols_per_row, figsize=(fig_width, fig_height), sharex=True, squeeze=False)
    fig.suptitle('各地区/城市比例历史数据与 ARIMA 预测 (2010-2035)', fontsize=16, y=0.99)

    try:
        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False
    except Exception as font_error:
        print(f"\n注意: 设置中文字体失败: {font_error}")
        print("图表标题和标签可能显示乱码。")

    axes_flat = axes.flatten()

    for i, loc in enumerate(locations):
        ax = axes_flat[i]
        if loc in df.columns:
             df[loc].plot(ax=ax, label='历史数据', marker='o', linestyle='-')
        if loc in forecast_df.columns and forecast_df[loc].notna().any():
            forecast_df[loc].plot(ax=ax, label='ARIMA 预测', linestyle='--', marker='x')
            if fitted_models.get(loc) is not None:
                try:
                    _, conf_int_plot = fitted_models[loc].predict(n_periods=forecast_horizon, return_conf_int=True)
                    conf_int_df = pd.DataFrame(conf_int_plot, index=forecast_index, columns=['lower', 'upper'])
                    conf_int_df['lower'] = conf_int_df['lower'].clip(lower=0)
                    ax.fill_between(conf_int_df.index.to_timestamp(),
                                    conf_int_df['lower'],
                                    conf_int_df['upper'],
                                    color='gray', alpha=0.2, label='95% 置信区间 (≥0)')
                except Exception as plot_ci_error:
                     print(f"绘制 {loc} 的置信区间时出错: {plot_ci_error}")
        ax.set_title(loc)
        ax.set_ylabel('比例 (%)')
        ax.legend(fontsize='small')
        ax.grid(True, linestyle=':', alpha=0.6)

    for j in range(i + 1, len(axes_flat)):
        axes_flat[j].set_visible(False)
    for row in range(n_rows):
        for col in range(cols_per_row):
            ax = axes[row, col]
            if ax.is_visible():
                if row == n_rows - 1:
                    ax.set_xlabel('年份')
                else:
                    ax.tick_params(labelbottom=False)

    plt.tight_layout(rect=[0, 0.03, 1, 0.96])
    plt.show()

共 36 个地区/城市需要处理。
开始为每个地区/城市拟合 ARIMA 模型并预测...

--- 正在处理: 北京 (1/36) ---
为 '北京' 找到的最佳 ARIMA 模型阶数: (0, 2, 0)
模型详细参数:
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                   12
Model:               SARIMAX(0, 2, 0)   Log Likelihood                 -33.683
Date:                Wed, 26 Mar 2025   AIC                             69.366
Time:                        13:00:30   BIC                             69.669
Sample:                    12-31-2010   HQIC                            69.034
                         - 12-31-2021                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2        49.3411     26.923      1.833      0.067      -3.427     102.109
Ljung-Box (L1) (Q)

In [39]:
import pandas as pd
import numpy as np
import pmdarima as pm
import warnings
import matplotlib.pyplot as plt # 可选，用于绘图
import os
import sys # <--- 新增导入
import contextlib # <--- 新增导入

warnings.filterwarnings("ignore") # 忽略 ARIMA 拟合中可能出现的警告

# --- 新增：定义标准输出重定向的上下文管理器 ---
@contextlib.contextmanager
def redirect_stdout_to_file(filename, mode='w', encoding='utf-8'):
    """一个临时重定向 sys.stdout 到文件的上下文管理器。"""
    original_stdout = sys.stdout
    try:
        with open(filename, mode, encoding=encoding) as f:
            sys.stdout = f
            yield f # 可以选择性地 yield 文件对象，虽然这里没用到
    finally:
        sys.stdout = original_stdout # 保证无论如何都恢复 stdout
# --- 结束新增部分 ---

# 1. 准备数据 (保持不变)
data = {
    '年份': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021],
'北京': [91.12, 85.19, 88.3, 79.79, 88.03, 87.57, 86.32, 80.39, 81.09, 90.84, 97.4, 98.98],
'天津': [87.19, 83.24, 82.62, 75.72, 77.12, 78.08, 74.91, 78.94, 88.76, 98.67, 104.01, 105.82],
'河北': [67.42, 71.11, 78.0, 76.29, 82.61, 83.1, 83.82, 85.96, 87.67, 90.33, 90.7, 87.23],
'山西': [70.3, 79.35, 84.7, 84.01, 87.18, 87.52, 89.91, 87.07, 89.85, 90.67, 92.22, 90.91],
'内蒙古': [61.09, 66.99, 77.32, 73.72, 87.15, 90.38, 95.29, 93.92, 95.21, 95.72, 93.47, 94.49],
'辽宁': [97.83, 98.17, 101.2, 102.5, 113.75, 116.98, 123.8, 114.38, 114.07, 104.41, 107.19, 97.38],
'吉林': [73.57, 71.09, 76.84, 74.55, 82.06, 90.09, 95.57, 88.38, 86.87, 86.1, 88.15, 82.61],
'黑龙江': [62.98, 64.98, 60.95, 63.5, 64.3, 65.39, 70.39, 68.2, 71.22, 71.79, 76.97, 75.12],
'上海': [88.07, 86.96, 90.19, 94.96, 100.9, 97.92, 94.0, 93.72, 95.17, 93.8, 92.89, 92.84],
'江苏': [100.68, 100.94, 105.62, 102.89, 110.02, 112.31, 113.17, 110.89, 109.36, 107.53, 105.25, 98.99],
'浙江': [109.14, 109.62, 106.31, 107.26, 107.54, 105.45, 104.36, 99.63, 96.46, 93.39, 90.4, 87.57],
'安徽': [50.3, 67.39, 73.72, 72.61, 81.41, 84.69, 87.43, 89.43, 90.99, 91.47, 89.23, 86.52],
'福建': [100.26, 104.65, 108.62, 107.09, 111.93, 111.86, 111.47, 106.54, 101.54, 95.54, 92.18, 86.85],
'江西': [80.38, 83.45, 87.03, 85.44, 91.3, 88.12, 89.9, 90.61, 93.05, 94.32, 88.73, 83.2],
'山东': [74.28, 77.39, 79.18, 77.45, 79.33, 79.85, 77.24, 81.64, 81.68, 81.74, 78.71, 82.5],
'河南': [87.77, 99.98, 115.06, 107.07, 119.88, 122.07, 122.83, 122.35, 116.76, 112.9, 106.46, 102.8],
'湖北': [85.31, 85.88, 94.65, 86.89, 91.42, 91.93, 91.74, 88.77, 89.38, 87.1, 84.88, 81.88],
'湖南': [71.94, 75.91, 84.22, 76.05, 83.4, 85.13, 85.76, 84.05, 84.76, 86.25, 87.17, 87.01],
'广东': [84.53, 88.96, 101.79, 94.29, 107.95, 111.42, 114.46, 113.68, 114.44, 124.02, 106.4, 100.44],
'广西': [70.74, 81.48, 97.58, 91.12, 105.04, 106.43, 107.61, 109.37, 107.74, 110.82, 108.77, 103.37],
'海南': [56.63, 72.31, 83.41, 78.5, 82.93, 88.69, 91.6, 92.81, 91.92, 96.17, 93.31, 91.59],
'重庆': [100.5, 104.77, 106.08, 107.18, 107.25, 104.06, 102.29, 100.02, 98.47, 97.12, 94.25, 91.97],
'四川': [93.14, 97.87, 108.15, 103.63, 108.72, 110.33, 108.95, 106.4, 105.63, 102.65, 97.95, 93.17],
'贵州': [58.44, 67.43, 82.82, 74.54, 91.0, 101.92, 108.53, 109.0, 107.27, 107.13, 107.85, 105.53],
'云南': [59.96, 63.48, 70.71, 67.51, 73.29, 74.6, 79.43, 81.12, 83.86, 91.63, 95.23, 97.31],
'西藏': [28.44, 43.03, 57.81, 52.56, 62.49, 67.38, 72.41, 82.02, 92.27, 95.93, 97.43, 96.69],
'陕西': [90.87, 103.25, 117.3, 113.79, 125.79, 127.88, 127.45, 122.74, 118.83, 117.06, 110.27, 120.78],
'甘肃': [43.64, 47.83, 62.2, 56.65, 74.46, 96.29, 100.26, 101.62, 100.31, 102.8, 105.6, 107.44],
'青海': [53.5, 61.94, 70.92, 67.53, 74.73, 81.45, 84.54, 86.52, 85.28, 89.32, 89.83, 87.59],
'宁夏': [53.5, 58.52, 66.81, 63.08, 72.88, 78.71, 88.22, 92.48, 93.65, 94.41, 93.85, 89.92],
'新疆': [63.46, 66.4, 71.58, 69.46, 79.29, 89.55, 136.9, 142.45, 136.85, 118.59, 103.63, 92.53],
'大连市': [109.4, 109.35, 109.23, 108.73, 108.15, 102.72, 99.2, 101.14, 97.27, 96.98, 91.14, 99.85],
'宁波市': [192.29, 197.39, 198.22, 196.96, 232.02, 279.7, 267.52, 217.48, 186.66, 183.85, 179.88, 179.05],
'厦门市': [151.89, 162.83, 172.58, 183.14, 182.79, 177.07, 170.93, 167.4, 155.47, 153.11, 146.2, 147.66],
'青岛市': [89.96, 102.93, 112.71, 112.65, 108.83, 108.34, 103.45, 106.53, 104.86, 103.89, 113.05, 121.38],
'深圳市': [285.58, 277.18, 284.98, 316.08, 303.68, 286.36, 263.83, 260.86, 246.73, 217.35, 191.39, 184.1]
}
df = pd.DataFrame(data)
df['年份'] = pd.to_datetime(df['年份'], format='%Y')
df.set_index('年份', inplace=True)
df.index = df.index.to_period('A')

# 2. 定义预测参数 (保持不变)
locations = list(data.keys())[1:]
forecast_horizon = 2035 - 2021

# --- 定义输出文件名 ---
output_log_filename = 'arima_processing_log.txt'
output_csv_filename = 'arima_forecast_2022_2035.csv'

# --- 在重定向前打印一条消息到控制台 ---
print(f"--- 开始处理，所有详细文本输出将写入文件: {output_log_filename} ---")
print(f"--- 最终预测表将导出到 CSV 文件: {output_csv_filename} ---")

# --- 使用上下文管理器重定向输出 ---
try:
    with redirect_stdout_to_file(output_log_filename, encoding='utf-8'):

        # 3. 循环为每个地区/城市进行 ARIMA 预测 (现在 print 会写入文件)
        forecast_results = {}
        fitted_models = {}

        print(f"共 {len(locations)} 个地区/城市需要处理。") # <-- 写入文件
        print("开始为每个地区/城市拟合 ARIMA 模型并预测...\n") # <-- 写入文件

        for i, loc in enumerate(locations):
            print(f"--- 正在处理: {loc} ({i+1}/{len(locations)}) ---") # <-- 写入文件
            series = df[loc]
            try:
                auto_model = pm.auto_arima(series, start_p=0, start_q=0, max_p=3, max_q=3, m=1, d=None, test='adf',
                                           seasonal=False, stepwise=True, suppress_warnings=True, error_action='ignore', trace=False)
                if auto_model is None or not hasattr(auto_model, 'order'):
                     raise ValueError("auto_arima未能找到合适的模型。")
                print(f"为 '{loc}' 找到的最佳 ARIMA 模型阶数: {auto_model.order}") # <-- 写入文件
                print("模型详细参数:") # <-- 写入文件
                print(auto_model.summary()) # <-- 写入文件 (summary 的输出也会被捕获)
                print("-" * 60) # <-- 写入文件
                fitted_models[loc] = auto_model
                forecast, conf_int = auto_model.predict(n_periods=forecast_horizon, return_conf_int=True)
                forecast_results[loc] = forecast
            except Exception as e:
                print(f"!!! 无法为 '{loc}' 拟合或预测 ARIMA 模型: {e}") # <-- 写入文件
                print("   可能是因为数据序列过短、模式不明显、数值过大/波动剧烈或auto_arima搜索失败。") # <-- 写入文件
                forecast_results[loc] = [np.nan] * forecast_horizon
                fitted_models[loc] = None
            finally:
                print("\n") # <-- 写入文件


        # 4. 整理并展示预测结果 (现在 print 会写入文件)
        forecast_index = pd.period_range(start=str(df.index[-1].year + 1), periods=forecast_horizon, freq='A')
        forecast_df = pd.DataFrame(forecast_results, index=forecast_index)
        forecast_df_formatted = forecast_df.round(2).astype(str) + '%'
        forecast_df_formatted = forecast_df_formatted.replace('nan%', 'N/A')

        print("\n\n--- ARIMA 模型预测结果 (2022-2035) ---") # <-- 写入文件

        # 为了确保 DataFrame 完整打印到文件，使用 to_string()
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 2000) # 设置足够宽的宽度
        print(forecast_df_formatted.to_string()) # <-- 将完整表格写入文件
        pd.reset_option('display.max_rows')
        pd.reset_option('display.max_columns')
        pd.reset_option('display.width')

        # 导出到 CSV (打印的确认信息也会写入 TXT 文件)
        try:
            # 使用 utf-8-sig 编码确保 Excel 正确识别中文和特殊字符
            forecast_df_formatted.to_csv(output_csv_filename, encoding='utf-8-sig')
            print(f"\n预测结果已成功导出到 CSV 文件: {os.path.abspath(output_csv_filename)}") # <-- 写入文件
        except Exception as e:
            print(f"\n!!! 导出 CSV 文件时出错: {e}") # <-- 写入文件

        # 打印绘图准备信息 (也会写入 TXT 文件)
        plot_results = True # 设置为 False 则不绘图
        if plot_results:
            print("\n--- 正在准备绘制结果图表 (图表将单独显示)... ---") # <-- 写入文件

    # --- 重定向结束，后续 print 会回到控制台 ---
    print(f"--- 处理完成，详细文本输出已写入: {os.path.abspath(output_log_filename)} ---")

except Exception as global_error:
    # 捕获在重定向期间可能发生的其他未预料错误
    print(f"!!! 脚本执行过程中发生严重错误: {global_error} !!!")
    # 尝试将错误信息追加到日志文件
    try:
        with open(output_log_filename, 'a', encoding='utf-8') as f_err:
             f_err.write(f"\n\n!!! 脚本执行过程中发生严重错误: {global_error} !!!\n")
    except:
        pass # 忽略记录错误时发生的错误

--- 开始处理，所有详细文本输出将写入文件: arima_processing_log.txt ---
--- 最终预测表将导出到 CSV 文件: arima_forecast_2022_2035.csv ---
--- 处理完成，详细文本输出已写入: /Users/annfengdeye/Desktop/3.26下午2点/arima_processing_log.txt ---
